# Variational Quantum Eigensolver

Алгоритм "Variational Quantum Eigensolver" - це гібридний алгоритм із використанням класичних і квантових обчислень.

В цьому алгоритмі класичний комп'ютер керує експериментальними параметрами, що контролюють підготовку квантового стану, потім квантовий комп’ютер реалізує цей стан із кубітами і обчислює його властивості.

## Терміни (всього 3)

Для початку введемо визначення трьох ключових термінів:

* [Хвильова функція](https://uk.wikipedia.org/wiki/Хвильова_функція) $|\Psi⟩$ (Wavefunction): математичний опис квантового стану.
* [Гамільтоніан](https://uk.wikipedia.org/wiki/Гамільтоніан) (Hamiltonian): квантовий енергетичний оператор, який описує загальну енергію квантової системи.
* [Квантові вентилі](https://uk.wikipedia.org/wiki/Квантовий_вентиль) (Quantum gates): операції, що виконуються над кубітами, маніпуляції квантовими станами. (Просто: аналогом квантових вентилів є базові операції в класичних алгоритмах - І, АБО, НІ і т.д.)

## Теоретична частина

Ось як виглядає використання VQE для обчислення за допомогою квантового комп’ютера:

1. Перетворення [молекулярного Гамільтоніана](https://pennylane.ai/qml/demos/tutorial_quantum_chemistry/) у кубітовий Гамільтоніан.

   Береться репрезентація взаємодій електронів в молекулі і встановлюється співвідношення з системою кубітів. Ви можете уявити взаємодію між електронними орбіталями в молекулах як про створення квантової заплутаності у системі кубітів. Чим більша молекула, яку ви намагаєтеся змоделювати, тим більше у вас електронних орбіталей, отже, тим більше кубітів вам потрібно.

   ![VQE Circuit](vqe_circuit.png "VQE") ([arXiv:1704.05018](https://arxiv.org/pdf/1704.05018.pdf))

2. Вибір "пробної хвильової функції" або випадкового початкового стану і кодування його на квантовому комп'ютері.

   Уявіть, що цей пробний стан - це здогадка щодо електронної конфігурації (оскільки ви ще не знаєте відповіді). Створюється квантовий стан на квантовому процесорі, який представляє конкретну версію хвильової функції, використовуючи комбінацію заплутаних вентилів, однокубітових вентилів та їх послідовності.

   <img src="./bloch_sphere.png" width="300px">

3. Оцінюється енергія пробного (початкового) стану.

   Це робиться шляхом вимірювання аспектів квантового стану, який ви створили на попередньому кроці. Враховуючи те, що ви знаєте про Гамільтоніан молекули, ви можете зв’язати це з енергією в молекулі для даної електронної конфігурації.

4. Ця енергія передається оптимізатору, який працює на класичному комп’ютері.

   Потім оптимізатор генерує новий набір параметрів, які створюють нову пробну хвильову функцію на квантовому комп'ютері з меншою енергією. Процес повторюється, поки енергія не зійдеться до найнижчого значення. Ця кінцева енергія відповідає рішенню енергії основного стану (Варіаційний принцип Рітца).

   $|\Phi⟩ = E_{ground}|\Phi⟩$

Кроки 2-4 повторюються для Гамільтоніанів, що відповідають різним енергіям. Тоді Гамільтоніан з найменшою енергією відповідає конфігурації рівноваги, і вуаля! Ви знаєте відповідь.

Здатність використовувати квантовий комп’ютер для виконання завдань, які зазвичай важкі для класичних комп’ютерів (наприклад, створення пробного стану та вимірювання його енергії), є важливою частиною того, чому такий підхід є настільки перспективним.

## Практична частина: імплементація VQE для квантового комп'ютера PennyLane

У цій частини ми програмно реалізуємо VQE для знаходження основного стану молекули водню $H_2$. Спочатку ми побудуємо Гамільтоніан молекули, потім квантову програму для підготовки початкового стану і функцію втрат для оцінки енергії ([Першоджерело](https://pennylane.ai/qml/demos/tutorial_vqe/)).


In [ ]:
%pip install pennylane
%pip install matplotlib
%pip install jax
%pip install optax

import pennylane as qml
import matplotlib.pyplot as plt
from jax import numpy as np
import jax
import optax

jax.config.update("jax_platform_name", "cpu")
jax.config.update('jax_enable_x64', True)

### Гамільтоніан

Перший крок - опис молекули. Для цього необхідно вказати список атомів і масив їх координат.

In [ ]:
symbols = ["H", "H"]
# symbols = ["H", "H", "H", "H"]

coordinates = np.array([0.0, 0.0, -0.6614, 0.0, 0.0, 0.6614])
# coordinates = np.array([-1.5, 0.0, 0.0, -0.5, 0.0, 0.0, 0.5, 0.0, 0.0, 1.5, 0.0, 0.0])

Для більш складних молекул структуру можна імпортувати із файла використовуючи функцію [read_structure()](https://docs.pennylane.ai/en/stable/code/api/pennylane.qchem.read_structure.html#pennylane.qchem.read_structure)

Тепер ми можемо побудувати Гамільтоніан:

In [ ]:
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates)

print("Number of qubits = ", qubits)
print("The Hamiltonian is ", H)

Вивід функції, це комбінація операторів Паулі і кількість кубітів, які необхідні для квантової симуляції.

Більш детально про побудову складних Гамільтоніанів: [Building molecular Hamiltonians](https://pennylane.ai/qml/demos/tutorial_quantum_chemistry/)

### Підключення квантового комп'ютера

In [ ]:
device = qml.device("lightning.qubit", wires=qubits)

### Пробна хвильова функція

У наступному кроці нам потрібно означити пробну хвильову функцію і закодувати її з допомогою кубітів у формі:

$$\vert \Psi(\theta) \rangle = \cos(\theta/2)~|1100\rangle -\sin(\theta/2)~|0011\rangle,$$

де $\theta$ - варіаційни параметр, який потрібно оптимізувати.

* Перша частина $|1100\rangle$ представляє стан, де два електрони
у молекулі знаходяться на найнижчих орбіталях, - [стан Гартрі-Фока](https://uk.wikipedia.org/wiki/Метод_Гартрі_—_Фока). В нотації Йордана-Вігнера ([Jordan-Wigner](https://en.wikipedia.org/wiki/Jordan–Wigner_transformation))
* Друга частина $|0011\rangle$ кодує подвійно-збуджений стан, де дві частки збуджені із кубітів 0, 1 до 2, 3.

### Квантові вентилі

Квантове коло (набір квантових вентилів) для підготовки пробного стану $\vert \Psi(\theta) \rangle$ схематично зображений на малюнку.

<img src="./sketch_circuit.png" width="600px">

На цьому рисунку квантовий вентиль $G^{(2)}$ відповідає операції подвійного збудження
[DoubleExcitation](https://docs.pennylane.ai/en/stable/code/api/pennylane.DoubleExcitation.html#pennylane.DoubleExcitation).

В PennyLane вона реалізована як [Поворот Ґівенса](https://uk.wikipedia.org/wiki/Поворот_Ґівенса), який зв'язує 4-кубітні стани $\vert 1100 \rangle$ і $\vert 0011 \rangle$.

### Квантова програма

Для імплементації квантової програми із PennyLane ми скористаємося вбудованою функцією `hf_state`, яка генерує вектор, що представляє стан Гартрі-Фока.


In [ ]:
electrons = 2
# electrons = 4
hf = qml.qchem.hf_state(electrons = electrons, orbitals = qubits)
print(hf)

[1 1 1 1 0 0 0 0]


Масив `hf` використовується в операції `BasisState` для ініціалізації квантових регістрів.

Потім ми використовуємо операцію `DoubleExcitation` на 4-х кубітах. І у наступному кроці
розраховуємо очікуване значення Гамільтоніана молекули для пробного стану - операція `expval`

Декоратор `qml.qnode` служить для означення нашого квантового кола `ansatz` як вершини у
графі гібридних обчислень із параметром $\theta$:


In [ ]:
@qml.qnode(device)
def ansatz(param, wires):
    qml.BasisState(hf, wires=wires)
    qml.DoubleExcitation(param, wires=[0, 2, 4, 6])
    qml.DoubleExcitation(param, wires=[1, 3, 5, 7])
    return qml.expval(H)

# @qml.qnode(device)
# def circuit(params, wires, s_wires, d_wires, hf_state):
#     qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
#     return qml.expval(H)

Тепер ми можемо визначити нашу функцію втрат як очікуване значення, що розраховується вище:

In [ ]:
def cost_fn(param):
    return ansatz(param, wires=range(qubits))

### Знаходження основного стану

Тепер ми використаємо нашу цільову функцію для знаходження основного стану молекули $H_2$.

Для початку означимо класичний оптимізатор, який використовує градієнтний спуск.

In [ ]:
max_iterations = 100
conv_tol = 1e-06

opt = optax.sgd(learning_rate=0.4)

Ми ініціалізуємо параметр $\theta = 0$, що відповідає стану Hartree-Fock.

In [ ]:
theta = np.array(0.)

# store the values of the cost function
energy = [cost_fn(theta)]

# store the values of the circuit parameter
angle = [theta]

opt_state = opt.init(theta)

for n in range(max_iterations):

    gradient = jax.grad(cost_fn)(theta)
    updates, opt_state = opt.update(gradient, opt_state)
    theta = optax.apply_updates(theta, updates)

    angle.append(theta)
    energy.append(cost_fn(theta))

    conv = np.abs(energy[-1] - energy[-2])

    if n % 2 == 0:
        print(f"Крок = {n},  Енергія = {energy[-1]:.8f} Ha")

    if conv <= conv_tol:
        break

print("\n" f"Кінцеве значення енергії = {energy[-1]:.8f} Ha")
print("\n" f"Оптимальне значення параметра = {angle[-1]:.4f}")

Крок = 0,  Енергія = -1.74159044 Ha
Крок = 2,  Енергія = -1.74215982 Ha
Крок = 4,  Енергія = -1.74220163 Ha
Крок = 6,  Енергія = -1.74220469 Ha

Кінцеве значення енергії = -1.74220469 Ha

Оптимальне значення параметра = -0.0349


### Графіки

Виведемо значення енергії і параметра $\theta$ на графіки:

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
fig.set_figheight(5)
fig.set_figwidth(12)

# Full configuration interaction (FCI) energy computed classically
E_fci = -1.136189454088

# Add energy plot on column 1
ax1 = fig.add_subplot(121)
ax1.plot(range(n + 2), energy, "go", ls="dashed")
ax1.plot(range(n + 2), np.full(n + 2, E_fci), color="red")
ax1.set_xlabel("Крок оптимізації", fontsize=13)
ax1.set_ylabel("Енергія (Hartree)", fontsize=13)
ax1.text(0.5, -1.1176, r"$E_\mathrm{HF}$", fontsize=15)
ax1.text(0, -1.1357, r"$E_\mathrm{FCI}$", fontsize=15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Add angle plot on column 2
ax2 = fig.add_subplot(122)
ax2.plot(range(n + 2), angle, "go", ls="dashed")
ax2.set_xlabel("Крок оптимізації", fontsize=13)
ax2.set_ylabel("Параметр $\\theta$ (rad)", fontsize=13)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.subplots_adjust(wspace=0.3, bottom=0.2)
plt.show()

### Результати

У нашому випадку VQE сходиться після десятка ітерацій, для більш складних проблем може вимагатися 1000-10000 ітерацій (shots).

Оптимальне значення параметра $\theta^* = 0.208$ визначає стан

$$\vert \Psi(\theta^*) \rangle = 0.994~\vert 1100 \rangle - 0.104~\vert 0011 \rangle,$$

який є основним станом молекули $H_2$.

## Посилання

* Abhinav Kandala et al., "Hardware-efficient Variational Quantum Eigensolver for Small Molecules and Quantum Magnets". [arXiv:1704.05018](https://arxiv.org/pdf/1704.05018.pdf)
* Alberto Peruzzo, Jarrod McClean et al., “A variational eigenvalue solver on a photonic quantum processor”. [Nature Communications 5, 4213 (2014)](https://www.nature.com/articles/ncomms5213?origin=ppub).
* Jacob T. Seeley, Martin J. Richard, Peter J. Love. “The Bravyi-Kitaev transformation for quantum computation of electronic structure”. [Journal of Chemical Physics 137, 224109 (2012)](https://aip.scitation.org/doi/abs/10.1063/1.4768229).
* Introduction to Quantum Chemistry | PennyLane Tutorial https://www.youtube.com/watch?v=khC0CCjxB7k
* Variational Quantum Eigensolver (VQE) | PennyLane Tutorial https://www.youtube.com/watch?v=qiRtUUZ5s9s
* Метод Рітца https://uk.wikipedia.org/wiki/Метод_Рітца